# Eksik Veri Doldurma Yöntemleri

Makine Öğrenmesinde veri temizliği önemlidir. Algoritmaların çoğu eksik veriye karşı hassastır, eksik veriyle çalışmak yanlı ve yanlış sonuçlar elde etmemize sebep olur.


pandas kütüphanesindeki isna() fonksiyonu verisetimizdeki NaN değerleri bulmamızı sağlar. Fonksiyon, boş olan değerlere True(1), diğerlerine False(0) atar. Böylece sum() fonksiyonunu ile birlikte kullanarak sütun bazında kaç tane boş değer olduğunu görebiliriz. Ard arda iki sum() fonksiyonunu kullanarak verisetindeki toplam boş değer sayısını öğrenebiliriz. 

In [24]:
import pandas as pd
data_path = "D:\\VeriAnaliziKitabı\\Bölüm1\\NBA_Dataset.csv"
df = pd.read_csv(data_path)

In [25]:
ncounts=pd.DataFrame([df.isna().sum()]).T


In [26]:
ncounts=ncounts.rename(columns={1:'data_missing'})
ncounts

,0
Team,0
Points_Scored,0
Weightlifting_Sessions_Average,9
Yoga_Sessions_Average,20
Laps_Run_Per_Practice_Average,14
Water_Intake,14
Players_Absent_For_Sessions,29


In [27]:
df.isna().sum().sum()

86

Verisetindeki boş değerli satırları sildiğimizi düşünürsek; 62 satır veri kaybederiz. Sütun bazında silersek 5 sütun veri kaybederiz.Dolayısıyla verileri yoketmek doğru bir yöntem değildir. 

In [28]:
df.shape, df.dropna(axis=0).shape

((159, 7), (97, 7))

In [29]:
df.shape, df.dropna(axis=1).shape

((159, 7), (159, 2))

##  Pandas ile Veri Düzeltme ##

Pandas, boş veri üç şekilde doldurabilir.
ffill bir sonraki değer ile, bfill ise bir önceki değer ile doldurur. groupby daha gelişmiş bir tekniktir. Boş olan veriyi, kendisiyle aynı gruptan olan verilere dayanarak doldurur.

In [20]:
#rename komutu ile uzun olan sütun isimlerini daha kısa isimlendirebiliriz.
df = df.rename(columns={"Weightlifting_Sessions_Average":"WL","Water_Intake":"WI","Yoga_Sessions_Average":"Yoga","Laps_Run_Per_Practice_Average":"Laps","Players_Absent_For_Sessions":"PAFS"})
#Belirli bir değerle boş değerler doldurulabilir.
df["WL"].fillna(-1)

0      23.0
1      24.0
2      24.0
3      26.0
4      27.0
       ... 
154    12.0
155    12.0
156    12.0
157    13.0
158    14.0
Name: WL, Length: 159, dtype: float64

WL sütununun ortalama veya medyan değerleri ile veya belirli bir değrle eksik değerleri doldurmamız mümkündür. Boş değerin bulunduğu satırdaki takıma ait ortalama ile eksik değeri doldurmak daha uygun bir çözümdür.

In [16]:
#Groupby team
mean_WL=df.groupby("Team")['WL'].mean().to_dict()

In [17]:
mean_WL

{'Bulls': 11.214285714285714,
 'Clippers': 29.0,
 'Lakers': 23173.645833333332,
 'Porcupines': 42.529411764705884,
 'Trailblazers': 25.425925925925924,
 'Warriors': 18.727272727272727}

In [18]:
for index, row in df.iterrows():
    team = row['Team']
    if pd.isna(row['WL']):
        mean_value = mean_WL.get(team)
        df.at[index, 'WL'] = mean_value

Satır satır gezerek boş değerlere takımın ortalama değerini atayabiliriz. Yukarıdaki kodu tek satırla lambda  ve transform fonksiyonları kullanarak da yazabiliriz.

In [19]:
df['WL']=df.groupby('Team')['WL'].transform(lambda x:x.fillna(x.mean()))

## Sklearn ile Eksik Veri Doldurma ##

scikit-learn kütüphanesindeki teknikeri kullanarak eksik verileri doldurmak mümkündür. En çok kullanılan teknikler şöyledir.

* SimpleImputer: Bu teknik ortalama, ortanca veya en sık kullanılan değerler ile boş değerleri doldurur.

* IterativeImputer: Öznitelikler arasında yüksek korelasyon olduğunda; adım adım eksik verileri doldurur.

* KNN imputation : Eksik veriyi belirlenen k değeri uzaklığındaki komşularının ortalama değeri ile doldurur. Bu tekniğin avantajı, makine öğrenmesi algoritmaları için önemli olan verideki korelasyonları korumasıdır. 

Özetle, scikit-learn kütüphenesi güçlü ve zayıf yönleri olan teknikler sunar, verisetine ve probleme uygun olan teknik seçilmelidir.

In [21]:
# 1 Simple Imputer

Features=['WL','Yoga','Laps','WI','PAFS']

from sklearn.impute import SimpleImputer
impt=SimpleImputer(strategy='mean')

#Fit & Transform

si_impt=impt.fit_transform(df[Features])
si_impt_df=pd.DataFrame(si_impt,columns=Features)

si_impt_df

,WL,Yoga,Laps,WI,PAFS
0,23.0,25.000000,30.0,5.0,4.000000
1,24.0,26.000000,31.0,5.0,8.000000
2,24.0,27.000000,31.0,5.0,7.000000
3,26.0,29.000000,34.0,5.0,1.000000
4,27.0,21.690647,34.0,5.0,6.000000
...,...,...,...,...,...
154,12.0,12.000000,13.0,1.0,4.415385
155,12.0,12.000000,14.0,1.0,4.415385
156,12.0,13.000000,14.0,1.0,4.415385
157,13.0,14.000000,15.0,1.0,6.000000


In [23]:
# KNN Imputer 

from sklearn.impute import KNNImputer

KNN=KNNImputer(n_neighbors=3)

#Fit & Transform

KNN_impt=KNN.fit_transform(df[Features])

KNN_impt_df=pd.DataFrame(KNN_impt,columns=Features)

KNN_impt_df

,WL,Yoga,Laps,WI,PAFS
0,23.0,25.000000,30.0,5.0,4.000000
1,24.0,26.000000,31.0,5.0,8.000000
2,24.0,27.000000,31.0,5.0,7.000000
3,26.0,29.000000,34.0,5.0,1.000000
4,27.0,22.666667,34.0,5.0,6.000000
...,...,...,...,...,...
154,12.0,12.000000,13.0,1.0,4.666667
155,12.0,12.000000,14.0,1.0,5.333333
156,12.0,13.000000,14.0,1.0,6.000000
157,13.0,14.000000,15.0,1.0,6.000000
